In [5]:
import multiprocessing
import numpy
import ifcopenshell
import ifcopenshell.util.placement
import ifcopenshell.util.element
import ifcopenshell.geom
import ifcopenshell.api.root
import ifcopenshell.api.unit
import ifcopenshell.api.context
import ifcopenshell.api.project
import ifcopenshell.api.geometry
import json
from typing import Dict, List, Tuple, Optional

In [2]:
def get_element_material(element) -> Dict:
    """Extract material information from an IFC element."""
    material_info = {"name": "Unknown", "properties": {}}
    
    # Get material relationships
    material_select = ifcopenshell.util.element.get_material(element)
    
    if material_select:
        if hasattr(material_select, 'Name'):
            material_info["name"] = material_select.Name or "Unnamed"
            
        # Try to get material properties if they exist
        if hasattr(material_select, 'MaterialProperties'):
            for props in material_select.MaterialProperties:
                if hasattr(props, 'Properties'):
                    for prop in props.Properties:
                        if hasattr(prop, 'Name') and hasattr(prop, 'NominalValue'):
                            material_info["properties"][prop.Name] = prop.NominalValue.wrappedValue
    
    return material_info

def get_element_size(element) -> Dict:
    """Extract size information from an IFC element."""
    size_info = {"length": None, "width": None, "height": None}
    
    # Get property sets
    psets = ifcopenshell.util.element.get_psets(element)
    
    # Look for common property set names that might contain dimensions
    for pset_name, properties in psets.items():
        # Check for length
        for length_key in ['Length', 'LENGTH', 'OverallLength']:
            if length_key in properties:
                size_info['length'] = properties[length_key]
                
        # Check for width
        for width_key in ['Width', 'WIDTH', 'OverallWidth']:
            if width_key in properties:
                size_info['width'] = properties[width_key]
                
        # Check for height
        for height_key in ['Height', 'HEIGHT', 'OverallHeight']:
            if height_key in properties:
                size_info['height'] = properties[height_key]
    
    # If no properties found, try to get from geometry
    if all(v is None for v in size_info.values()):
        try:
            settings = ifcopenshell.geom.settings()
            shape = ifcopenshell.geom.create_shape(settings, element)
            bbox = shape.bbox()
            size_info['length'] = bbox.diagonal
            size_info['width'] = bbox.xmax - bbox.xmin
            size_info['height'] = bbox.zmax - bbox.zmin
        except:
            pass
    
    return size_info



In [19]:
def analyze_clashes(model: ifcopenshell.file, tree: ifcopenshell.geom.tree) -> Dict:
    """Analyze clashes between beams and columns, including material and size information."""
    beams = model.by_type("IfcBeam")
    columns = model.by_type("IfcColumn")
    # Detect clashes
    columns_to_beams_clashes = tree.clash_collision_many(
        columns, beams, 
        allow_touching=True
    )
    # Prepare clash data
    clash_data = {
        "total_clashes": len(columns_to_beams_clashes),
        "clashes": []
    }
    
    # Analyze each clash
    for i, collision in enumerate(columns_to_beams_clashes):
        column = model.by_id(collision.a.id())
        beam = model.by_id(collision.b.id())
        clash_info = {
            "clash_id": f"clash_{i}",
            "location": {
                "x": collision.p1[0],
                "y": collision.p1[1],
                "z": collision.p1[2]
            },
            "column": {
                "id": column.id(),
                "global_id": column.GlobalId,
                "material": get_element_material(column),
                "size": get_element_size(column)
            },
            "beam": {
                "id": beam.id(),
                "global_id": beam.GlobalId,
                "material": get_element_material(beam),
                "size": get_element_size(beam)
            }
        }
        
        clash_data["clashes"].append(clash_info)
    
    return clash_data, columns_to_beams_clashes

In [20]:
def main():
    # Input and output file paths
    ifc_file_path = 'Dummy_Detailed_Fixed.ifc'
    ifc_output_path = 'DummyModelFixed_collided.ifc'
    json_output_path = 'clash_analysis.json'
    
    try:
        # Load the IFC file
        model = ifcopenshell.open(ifc_file_path)
        
        # Create a geometric tree
        tree = ifcopenshell.geom.tree()
        
        # Set up 3D context for visualization
        model3d = ifcopenshell.api.context.add_context(model, context_type="Model")
        body = ifcopenshell.api.context.add_context(model, context_type="Model", 
                                                  context_identifier="Body", 
                                                  target_view="MODEL_VIEW", 
                                                  parent=model3d)
        
        # Define pyramid geometry for clash visualization
        vertices = [[(0.,0.,.5), (0.,.2,.5), (.2,.2,.5), (.2,0.,.5), (.1,.1,0.)]]
        faces = [[(0,1,2,3), (0,4,1), (1,4,2), (2,4,3), (3,4,0)]]
        representation = ifcopenshell.api.geometry.add_mesh_representation(
            model, context=body, vertices=vertices, faces=faces)
        
        # Process geometry
        settings = ifcopenshell.geom.settings()
        elements = model.by_type("IfcBeam") + model.by_type("IfcColumn")
        iterator = ifcopenshell.geom.iterator(settings, model, 
                                            multiprocessing.cpu_count(), 
                                            include=elements)
        
        if iterator.initialize():
            while True:
                tree.add_element(iterator.get())
                if not iterator.next():
                    break
        
        # Analyze clashes and get visualization data
        clash_data, columns_to_beams_clashes = analyze_clashes(model, tree)
        
        # Create visualization elements for clashes
        matrix = numpy.eye(4)
        for collision in columns_to_beams_clashes:
            matrix[:,3][0:3] = list(collision.p1)
            element = ifcopenshell.api.root.create_entity(model, ifc_class="IfcWall")
            ifcopenshell.api.geometry.edit_object_placement(model, product=element, 
                                                          matrix=matrix)
            ifcopenshell.api.geometry.assign_representation(model, product=element, 
                                                          representation=representation)
        
        # Save the modified IFC file with clash visualizations
        model.write(ifc_output_path)
        
        # Save clash analysis to JSON file
        with open(json_output_path, 'w', encoding='utf-8') as f:
            json.dump(clash_data, f, indent=2)
        
        print(f"Analysis complete. Found {clash_data['total_clashes']} clashes.")
        print(f"Results saved to {json_output_path}")
        print(f"Visualizations saved to {ifc_output_path}")
        
    except Exception as e:
        print(f"Error during analysis: {str(e)}")

if __name__ == "__main__":
    main()

Analysis complete. Found 162 clashes.
Results saved to clash_analysis.json
Visualizations saved to DummyModelFixed_collided.ifc
